<a href="https://colab.research.google.com/github/elmimansur/NN-Practices/blob/main/try1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [ ]:
import tensorflow as tf
from tensorflow import keras

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#Reshape
x_train2 = x_train.reshape(60000,784)
x_test2 = x_test.reshape(10000,784)

#One-hot-encoded versions of the labels
y_train2 = keras.utils.to_categorical(y_train, num_classes=10)
y_test2 = keras.utils.to_categorical(y_test, num_classes=10)

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
import numpy as np
import math   

class NeuralNetwork:
    def __init__(self, lr):
        dimension = 100
        self.lr = lr
        self.losses = []
         
    def sigmoid(self,x):
        return np.array(1.0/(1+ np.exp(-x)), dtype=np.float32)
    def d_sigmoid (self,x):
        return self.sigmoid(x) * (1.0 - self.sigmoid(x))
        
    def softmax(self,x):
        exps = np.exp(x- x.max())
        return exps / np.sum(exps, axis=0)
    def d_softmax(self,x):
        exps = np.exp(x- x.max())
        return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))

    def feedforward(self,x):
        self.x = np.expand_dims(x,axis =1)
        self.layer1 =  self.weights1 @ self.x + self.bias1
        self.output1 = self.sigmoid(self.layer1)
        
        self.layer2 =  self.weights2 @ self.output1   + self.bias2
        self.output= self.softmax(self.layer2)
        #self.output=np.squeeze(self.output)

    def backprop(self):
        d_weights1 = np.zeros(self.weights1.shape)
        d_weights2 = np.zeros(self.weights2.shape)
        
        d_bias1    = np.zeros(self.bias1.shape)
        d_bias2    = np.zeros(self.bias2.shape)
        loss = 0
        for j in range(len(self.y)):
            single_input = self.input[j]
            self.feedforward(single_input)
            d_loss = (self.output - self.y[j])/len(self.y)
            loss+= (self.output - self.y[j])**2/len(self.y)
            
            delta2 = d_loss.T @ self.d_softmax(self.layer2)
            d_weights2 += delta2 @ self.output1.T
            d_bias2+= delta2
            
            dA1 =  self.weights2.T @ d_loss
            delta1 = dA1 @ self.d_sigmoid(self.layer1)
            d_weights1 += delta1 @  self.x.T 
            d_bias1+= delta1
            
            
        #update for gradient step
        self.weights1 -= self.lr*d_weights1
        self.weights2 -= self.lr*d_weights2
        
        self.bias1    -= self.lr*d_bias1
        self.bias2    -= self.lr*d_bias2
        
        self.loss = loss

    def initial(self,x,y,n_iter):
        dimension = 100
        self.input = x
        self.weights1 = np.random.rand(dimension, x.shape[1])
        self.weights2 = np.random.rand(10,dimension)
        
        self.bias1 = np.random.rand(dimension,1)
        self.bias2 = np.random.rand(10,1)
        
        self.y            = y
        for j in range(n_iter):
            self.backprop()
            self.losses.append(self.loss)
    
    def predict(self,x):
        self.feedforward(x)
        return self.output

In [ ]:
nn = NeuralNetwork(lr=0.001)
nn.initial(x_train2,y_train2,n_iter=10)

In [ ]:
predicted = np.zeros(y_test2.shape)
for i in range(len(predicted)):
    predicted[i] = nn.predict(np.array(x_test2[i]))

predicted_class = np.round(predicted)

# Calculate correct matches ,, max almak lazım
match_count = sum([int((y == y_).all()) for y, y_ in zip(y_test2, predicted_class)])

# Calculate the accuracy
accuracy = match_count / len(y_test2)
# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.098


In [ ]:
## dersteki örnek kod: 

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
import math

def sigmoid(x):
    return np.array(1.0/(1+ np.exp(-x)), dtype=np.float32)
    
def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

class NeuralNetwork:
    def __init__(self, lr):
        dimension = 1
        self.lr = lr
        self.activation   = sigmoid
        self.d_activation = sigmoid_derivative
        self.losses = []

    def feedforward(self,x):
        self.x = np.expand_dims(x,axis =1)
        self.layer1_pa = self.weights1 @  self.x + self.bias1
        self.output = self.activation(self.layer1_pa)

    def backprop(self):
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        d_weights1 = np.zeros(self.weights1.shape)
        d_bias1    = np.zeros(self.bias1.shape)
        loss = 0
        for j in range(len(self.y)):
            # compute gradient per each input image
            single_input = self.input[j]
            self.feedforward(single_input)
            d_loss = (self.output - self.y[j])/len(self.y)

            #softmax'i loss'a /binary cross
            loss+= (self.output - self.y[j])**2/len(self.y)#(self.y[j]*self.output + (1-self.y[j])*(1-self.output))**2/len(self.y)

            delta = d_loss @ self.d_activation(self.layer1_pa)

            d_weights1 +=  delta @ self.x.T
            d_bias1    += delta
        #taking the gradient step with learning rate lr per point
        self.weights1 -= self.lr*d_weights1
        self.bias1    -= self.lr*d_bias1
        self.loss = loss

    def fit(self,x,y,n_iter):
        dimension = 1
        self.input        = x
        self.weights1     = np.random.rand(dimension, x.shape[1])
        self.bias1        = np.random.rand(dimension,1)      
        self.y            = y
        for j in range(n_iter):
            self.backprop()
            self.losses.append(self.loss)
    
    def predict(self,x):
        self.feedforward(x)
        return self.output

In [ ]:
nn = NeuralNetwork(lr=0.001)
nn.fit(x_train2,y_train2,n_iter=10)

ValueError: ignored